In [170]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import configparser as configparser
from ipywidgets import widgets

In [171]:
config = configparser.ConfigParser()
config.read('../config.ini')
mapbox_token = config['mapbox']['secret_token']

In [172]:
df = pd.read_csv('inno_geocoded.csv')
df = df.dropna(subset=['lng', 'lat'])

df['innoint'] = df.inno_nr.str.strip('inno_').astype('int')
df['date'] = pd.to_datetime(df['created_at']).dt.date

df.head(3)

/tmp/ipykernel_27427/1224049548.py:1: DtypeWarning:

Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.



,created_at,twitter_id,user_id,user_location,longitude,latitude,place_box_1_long,place_box_1_lat,place_box_2_long,place_box_2_lat,...,quarter,suburb,allotments,farm,hamlet,isolated_dwelling,island,time,innoint,date
0,2012-01-01 08:10:59+00:00,153387731521773569,191216844,Paraguay,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"01/01/2012, 08:10:59",2,2012-01-01
2,2012-01-03 14:15:36+00:00,154204266041585664,298356610,bogota,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"03/01/2012, 14:15:36",2,2012-01-03
3,2012-01-08 02:41:19+00:00,155841483243663360,285206171,Ibagué,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"08/01/2012, 02:41:19",2,2012-01-08


In [173]:
def process_geocodedtwitter_data(df, zone = 'country_y', inno_choose=inno_choose):

    # Choose Innovation
    df = df[df.inno_nr == inno_choose]
    df = df.reset_index()

    # Saving countries positions (latitude and longitude per subzones)
    country_position = df[[f'{zone}', 'lat', 'lng']].drop_duplicates([f'{zone}']).set_index([f'{zone}'])

    # Pivoting per category
    cats = list(df.inno_nr.unique())
    cats.sort()
    
    df = pd.pivot_table(df, values='innoint', index=['date', f'{zone}'], columns=['inno_nr'])
    df.columns = cats

    # Merging locations after pivoting
    df = df.join(country_position)

    # Filling nan values with 0
    df = df.fillna(0)

    # Compute bubble sizes
    df['size'] = df[inno_choose]

    # Compute bubble color
    df['color'] = 'fuchsia' #df[cats].sum(axis=1)
    
    return df

In [174]:
df = process_geocodedtwitter_data(df)
df.head(10)

inno_01        lat         lng  size    color
date       country_y                                               
2013-04-01 Guatemala        1  15.687101  -90.122655     1  fuchsia
2013-04-03 México           1  23.658512 -102.007710     1  fuchsia
2013-04-04 España           1  39.491522   -0.473903     1  fuchsia
2013-04-05 Venezuela        1  10.237514  -67.589022     1  fuchsia
2013-05-03 France           1  44.763984    4.915850     1  fuchsia
2013-05-08 México           1  23.658512 -102.007710     1  fuchsia
2013-06-03 中国               1  43.813074  125.317122     1  fuchsia
2013-06-05 Argentina        1 -33.123759  -64.348978     1  fuchsia
           España           1  39.491522   -0.473903     1  fuchsia
2013-06-06 Argentina        1 -33.123759  -64.348978     1  fuchsia

In [175]:
day = df.index[-1][0]
tmp = df.xs(day)
tmp

,inno_01,lat,lng,size,color
country_y,,,,,
Venezuela,1,10.237514,-67.589022,1,fuchsia


import plotly.express as px

fig = px.scatter_mapbox(tmp, lat="lat", lon="lng", hover_name="color", hover_data=["size"],
                        color = "color",
                        size = 'size',
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        zoom=3, height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


# Display the figure
fig.show()


In [184]:
# Selecting the day to display
#day = '2012-01-01'
#tmp = df#.xs(day)

# Create the figure and feed it all the prepared columns
fig = go.Figure(
    go.Scattermapbox(
        lat=tmp['lat'],
        lon=tmp['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=tmp['size']*10,
            color='fuchsia',
            showscale=True,
            colorbar={'title':'Innovations', 'titleside':'top', 'thickness':4, 'tickprefix':' Inno Nr.: '}
        )
    )
)
# Specify layout information
fig.update_layout(
    mapbox=dict(
        accesstoken=mapbox_token,
        center=go.layout.mapbox.Center(lat=7, lon=-33),
        zoom=1,
        style="open-street-map"
    ),

)

# Display the figure
fig.show()

In [177]:
df


,,inno_01,lat,lng,size,color
date,country_y,,,,,
2013-04-01,Guatemala,1,15.687101,-90.122655,1,fuchsia
2013-04-03,México,1,23.658512,-102.007710,1,fuchsia
2013-04-04,España,1,39.491522,-0.473903,1,fuchsia
2013-04-05,Venezuela,1,10.237514,-67.589022,1,fuchsia
2013-05-03,France,1,44.763984,4.915850,1,fuchsia
...,...,...,...,...,...,...
2019-07-02,Colombia,1,3.451792,-76.532494,1,fuchsia
2019-07-05,Argentina,1,-33.123759,-64.348978,1,fuchsia
2019-08-03,Colombia,1,3.451792,-76.532494,1,fuchsia


In [178]:
days = df.index.levels[0].tolist()

In [179]:
frames = [{   
    'name':'frame_{}'.format(day),
    'data':[{
        'type':'scattermapbox',
        'lat':df.xs(day)['lat'],
        'lon':df.xs(day)['lng'],
        'marker':go.scattermapbox.Marker(
            size=df.xs(day)['size'],
            color=df.xs(day)['color'],
            showscale=True,
            colorbar={'title':'Innovations', 'titleside':'top', 'thickness':4, 'tickprefix':' Inno Nr.: '},
        )#,
        #'customdata':np.stack((df.xs(day)['confirmed_display'], df.xs(day)['recovered_display'],  df.xs(day)['deaths_display'], pd.Series(df.xs(day).index)), axis=-1),
        #'hovertemplate': "<extra></extra><em>%{customdata[3]}  </em><br>🚨  %{customdata[0]}<br>🏡  %{customdata[1]}<br>⚰️  %{customdata[2]}",
    }],           
} for day in days]  

In [180]:
sliders = [{
    'transition':{'duration': 0},
    'x':0.08, 
    'len':0.88,
    'currentvalue':{'font':{'size':15}, 'prefix':'📅 ', 'visible':True, 'xanchor':'center'},  
    'steps':[{'label':str(day),
            'method':'animate',
            'args':[
                ['frame_{}'.format(day)],
                {'mode':'immediate', 'frame':{'duration':100, 'redraw': True}, 'transition':{'duration':50}}
              ],
        } for day in days]
}]



In [181]:
play_button = [{
    'type':'buttons',
    'showactive':True,
    'x':0.045, 'y':-0.08,
    'buttons':[{ 
        'label':'🎬', # Play
        'method':'animate',
        'args':[
            None,
            {
                'frame':{'duration':100, 'redraw':True},
                'transition':{'duration':50},
                'fromcurrent':True,
                'mode':'immediate',
            }
        ]
    }]
}]

In [182]:
# Defining the initial state
data = frames[0]['data']

# Adding all sliders and play button to the layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=play_button,
    mapbox={
        'accesstoken':mapbox_token,
        'center':{'lat':7, 'lon':-33},
        'zoom':1.7,
        'style':"open-street-map",
        
    }
)

# Creating the figure
fig = go.Figure(data=data, layout=layout, frames=frames)

# Displaying the figure
fig.show()